In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv


In [2]:
!pip install ycimpute

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, metrics, model_selection, svm
import missingno as msno
from ycimpute.imputer import iterforest,EM
from fancyimpute import KNN
from sklearn.preprocessing import OrdinalEncoder

import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from warnings import filterwarnings
filterwarnings('ignore')

pd.set_option('display.max_columns', None)
import gc

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


Using TensorFlow backend.


In [3]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage2(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


In [4]:
encoder=OrdinalEncoder()
imputer=KNN()

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

In [5]:
Ktest_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
Ktest_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
Ktrain_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
Ktrain_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")

In [6]:
Ktrain= pd.merge(Ktrain_transaction, Ktrain_identity, on='TransactionID', how='left', left_index=True, right_index=True)
Ktest= pd.merge(Ktest_transaction, Ktest_identity, on='TransactionID', how='left', left_index=True, right_index=True)


In [7]:
#print(Ktrain.shape, Ktest.shape)
#print(Ktest.info())
#print(Ktrain.info())

In [8]:
Ktrain_cat=Ktrain.select_dtypes(include='object')
Ktest_cat =Ktest.select_dtypes(include='object')

In [9]:
#Ktrain_cat.nunique()

In [10]:
#Ktest_cat.nunique()

In [11]:
Ktrain_cat1=Ktrain_cat.drop(['P_emaildomain','R_emaildomain','id_30','id_31','id_33','DeviceInfo'], axis=1)
Ktest_cat1=Ktest_cat.drop(['P_emaildomain','R_emaildomain','id-30','id-31','id-33','DeviceInfo'], axis=1)

In [12]:
for i in Ktrain_cat1:
    encode(Ktrain[i])
for i in Ktest_cat1:
    encode(Ktest[i])

In [13]:
Ktrain_cat2=pd.concat([Ktrain['P_emaildomain'],Ktrain['R_emaildomain'],Ktrain['id_30'],Ktrain['id_31'],Ktrain['id_33'],Ktrain['DeviceInfo']], axis=1)
Ktest_cat2=pd.concat([Ktest['P_emaildomain'],Ktest['R_emaildomain'],Ktest['id-30'],Ktest['id-31'],Ktest['id-33'],Ktest['DeviceInfo']], axis=1)


In [14]:
for i in Ktrain_cat2:
    encode(Ktrain[i])
for i in Ktest_cat2:
    encode(Ktest[i])

In [15]:
import gc
del Ktest_identity
del Ktest_transaction
del Ktrain_identity
del Ktrain_transaction
del Ktrain_cat1
del Ktest_cat1
del Ktrain_cat2
del Ktest_cat2
gc.collect()

62

In [16]:
Ktest = reduce_mem_usage2(Ktest)
Ktrain = reduce_mem_usage2(Ktrain)

Memory usage of dataframe is 1673.87 MB
Memory usage after optimization is: 458.70 MB
Decreased by 72.6%
Memory usage of dataframe is 1955.37 MB
Memory usage after optimization is: 526.13 MB
Decreased by 73.1%


In [17]:
Ktrain.shape

(590540, 434)

In [18]:
Ktest.shape

(506691, 433)

In [19]:
Ktest.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.953125,4.0,10409,111.0,150.0,3.0,226.0,2.0,170.0,87.0,1.0,NaN,16.0,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,1.0,1.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,47.950001,0.000000,0.0000,47.950001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.0000,47.950001,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,1.0,27.0,NaN,1.0,1.0,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,45.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,1.0,924.0
1,3663550,18403263,49.000000

In [20]:
z= Ktest.loc[:,'id-01':'id-38'].columns.str.replace('-','_')

In [21]:
z=list(z)
z

['id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38']

In [22]:
for x,y in zip(Ktest.loc[:,'id-01':'id-38'].columns, z):
    Ktest[y]=Ktest[x]
    del Ktest[x]
gc.collect()

64

* #### fill nulls / train test split

In [23]:
y=Ktrain["isFraud"]
X=Ktrain.drop(["isFraud", "TransactionID"], axis=1).astype('float64')
X= X.fillna(-999)

Ktest_id = Ktest['TransactionID']
X_Ktest = Ktest.drop(['TransactionID'], axis=1).astype('float64')
X_Ktest = X_Ktest.fillna(-999)

X_Ktest = X_Ktest[X.columns]

In [24]:
# Ktest= Ktest[Ktrain.columns]

* PCA Analysis

In [25]:
'''
X_fit=StandardScaler().fit_transform(X)
X_pca=PCA().fit(X_fit)
plt.plot(np.cumsum(X_pca.explained_variance_ratio_))
plt.title('All columns included', color='gray')
plt.xlabel("Number of Component", color='green')
plt.ylabel("Cumulative Variance Ratio", color='green')
plt.grid(color='gray', linestyle='-', linewidth=0.3)
plt.show()

X_Ktest_fit=StandardScaler().fit_transform(X_Ktest)
X_Ktest_pca=PCA().fit(X_Ktest_fit)
plt.plot(np.cumsum(X_Ktest_pca.explained_variance_ratio_))
plt.title('All columns included', color='gray')
plt.xlabel("Number of Component", color='green')
plt.ylabel("Cumulative Variance Ratio", color='green')
plt.grid(color='gray', linestyle='-', linewidth=0.3)
plt.show()
'''

'\nX_fit=StandardScaler().fit_transform(X)\nX_pca=PCA().fit(X_fit)\nplt.plot(np.cumsum(X_pca.explained_variance_ratio_))\nplt.title(\'All columns included\', color=\'gray\')\nplt.xlabel("Number of Component", color=\'green\')\nplt.ylabel("Cumulative Variance Ratio", color=\'green\')\nplt.grid(color=\'gray\', linestyle=\'-\', linewidth=0.3)\nplt.show()\n\nX_Ktest_fit=StandardScaler().fit_transform(X_Ktest)\nX_Ktest_pca=PCA().fit(X_Ktest_fit)\nplt.plot(np.cumsum(X_Ktest_pca.explained_variance_ratio_))\nplt.title(\'All columns included\', color=\'gray\')\nplt.xlabel("Number of Component", color=\'green\')\nplt.ylabel("Cumulative Variance Ratio", color=\'green\')\nplt.grid(color=\'gray\', linestyle=\'-\', linewidth=0.3)\nplt.show()\n'

In [26]:
# PCA Analysis is misleading here because we filled missing data. 

"""
#Final Model for Ktrain
X_pca = PCA(n_components=100).fit(X_fit)
X_fit = X_pca.fit_transform(X_fit)
X_pca.explained_variance_ratio_

#Final Model for Ktest

X_Ktest_pca = PCA(n_components=100).fit(X_Ktest_fit)
X_Ktest_fit = X_Ktest_pca.fit_transform(X_Ktest_fit)
print(X_pca.explained_variance_ratio_.sum())
print(X_Ktest_pca.explained_variance_ratio_.sum())
"""

'\n#Final Model for Ktrain\nX_pca = PCA(n_components=100).fit(X_fit)\nX_fit = X_pca.fit_transform(X_fit)\nX_pca.explained_variance_ratio_\n\n#Final Model for Ktest\n\nX_Ktest_pca = PCA(n_components=100).fit(X_Ktest_fit)\nX_Ktest_fit = X_Ktest_pca.fit_transform(X_Ktest_fit)\nprint(X_pca.explained_variance_ratio_.sum())\nprint(X_Ktest_pca.explained_variance_ratio_.sum())\n'

In [27]:
'''
xlist=[]
for i in range(1,101):
    xlist.append("a"+str(i))

X_new= pd.DataFrame(data=X_fit, columns= xlist)
X_Ktest_new= pd.DataFrame(data=X_Ktest_fit, columns= xlist)
X_Ktest_new.head()
'''

'\nxlist=[]\nfor i in range(1,101):\n    xlist.append("a"+str(i))\n\nX_new= pd.DataFrame(data=X_fit, columns= xlist)\nX_Ktest_new= pd.DataFrame(data=X_Ktest_fit, columns= xlist)\nX_Ktest_new.head()\n'

* train_test_split

In [28]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.25, random_state=42)

In [29]:
X = reduce_mem_usage2(X)
X_Ktest = reduce_mem_usage2(X_Ktest)

Memory usage of dataframe is 1946.36 MB
Memory usage after optimization is: 538.40 MB
Decreased by 72.3%
Memory usage of dataframe is 1670.00 MB
Memory usage after optimization is: 469.69 MB
Decreased by 71.9%


In [30]:
gc.collect()

20

### ML Algorithms

In [31]:
print(Ktrain.shape)
print(Ktest.shape)
print(X.shape)
print(X_Ktest.shape)


(590540, 434)
(506691, 433)
(590540, 432)
(506691, 432)


In [32]:
# It gets so much time to run with each models, so passed for now.

'''models = [LogisticRegression,
          KNeighborsClassifier,
          GaussianNB,
          SVC,
          DecisionTreeClassifier,
          RandomForestClassifier,
          GradientBoostingClassifier,
          LGBMClassifier,
          XGBClassifier
          #CatBoostClassifier
         ]

def compML (df, y, algorithm):
    
    #y=df[y]
    #X=df.drop(["PassengerId","Survived"], axis=1).astype('float64')
    #X_train, X_test,y_train,y_test=train_test_split(X,y, test_size=0.25, random_state=42)
    
    model=algorithm().fit(X_train, y_train)
    y_pred=model.predict(X_test)
    accuracy= accuracy_score(y_test, y_pred)
    #return accuracy
    model_name= algorithm.__name__
    print(model_name,": ",accuracy)
    
    
for i in models:
    compML(X,"isFraud",i)
    
'''

'models = [LogisticRegression,\n          KNeighborsClassifier,\n          GaussianNB,\n          SVC,\n          DecisionTreeClassifier,\n          RandomForestClassifier,\n          GradientBoostingClassifier,\n          LGBMClassifier,\n          XGBClassifier\n          #CatBoostClassifier\n         ]\n\ndef compML (df, y, algorithm):\n    \n    #y=df[y]\n    #X=df.drop(["PassengerId","Survived"], axis=1).astype(\'float64\')\n    #X_train, X_test,y_train,y_test=train_test_split(X,y, test_size=0.25, random_state=42)\n    \n    model=algorithm().fit(X_train, y_train)\n    y_pred=model.predict(X_test)\n    accuracy= accuracy_score(y_test, y_pred)\n    #return accuracy\n    model_name= algorithm.__name__\n    print(model_name,": ",accuracy)\n    \n    \nfor i in models:\n    compML(X,"isFraud",i)\n    \n'

* Light GBM

In [33]:
model= LGBMClassifier().fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9779252887187997

In [34]:
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


### Kaggle Test Data

In [35]:
# Ktest_pred= lgb_model.predict(X_Ktest_new)

In [36]:
predictions=model.predict(X_Ktest)
output=pd.DataFrame({"TransactionID":Ktest_id, "isFraud":predictions})
output.to_csv('submission_model.csv', index=False)